# Guided Capstone Step 4. Pre-Processing and Training Data Development

**The Data Science Method**  


1.   Problem Identification 


2.   Data Wrangling 
  
 
3.   Exploratory Data Analysis   

4.   **Pre-processing and Training Data Development**  
 * Create dummy or indicator features for categorical variables
  * Standardize the magnitude of numeric features
  * Split into testing and training datasets
  * Apply scaler to the testing set
5.   Modeling 
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

**<font color='teal'> Start by loading the necessary packages as we did in step 3 and printing out our current working directory just to confirm we are in the correct project directory. </font>**

In [1]:
#load python packages
import os
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [2]:
os.getcwd()

'D:\\Data Science\\JupyterNotebooks'

In [3]:
path="D:/Data Science/06 Applying the Data Science Method/Step 2/data"
os.chdir(path)

**<font color='teal'>  Load the csv file you created in step 3, remember it should be saved inside your data subfolder and print the first five rows.</font>**

In [4]:
df = pd.read_csv('step3_output.csv')
df.head()

,Name,state,summit_elev,vertical_drop,trams,fastEight,fastSixes,fastQuads,quad,triple,...,SkiableTerrain_ac,Snow Making_ac,daysOpenLastYear,yearsOpen,averageSnowfall,AdultWeekday,AdultWeekend,projectedDaysOpen,NightSkiing_ac,clusters
0,Eaglecrest Ski Area,Alaska,2600,1540,0,0,0,0,0,0,...,640,60,45,44,350.0,47.0,53.00000,90,0,2
1,Hilltop Ski Area,Alaska,2090,294,0,0,0,0,0,1,...,30,30,150,36,69.0,30.0,34.00000,152,30,0
2,Sunrise Park Resort,Arizona,11100,1800,0,0,0,1,2,3,...,800,80,115,49,250.0,74.0,78.00000,104,80,1
3,Yosemite Ski & Snowboard Area,California,7800,600,0,0,0,0,0,1,...,88,0,110,84,300.0,47.0,47.00000,107,0,1
4,Boreal Mountain Resort,California,7700,500,0,0,0,1,1,3,...,380,200,150,54,400.0,49.0,64.16681,150,200,1


## Create dummy features for categorical variables

**<font color='teal'> Create dummy variables for `state`. Add the dummies back to the dataframe and remove the original column for `state`. </font>**

Hint: you can see an example of how to execute this in Aiden's article on preprocessing [here](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-pre-processing-and-training-data-development-fd2d75182967). 

In [5]:
df.dtypes

Name                  object
state                 object
summit_elev            int64
vertical_drop          int64
trams                  int64
fastEight              int64
fastSixes              int64
fastQuads              int64
quad                   int64
triple                 int64
double                 int64
surface                int64
total_chairs           int64
Runs                   int64
TerrainParks           int64
LongestRun_mi        float64
SkiableTerrain_ac      int64
Snow Making_ac         int64
daysOpenLastYear       int64
yearsOpen              int64
averageSnowfall      float64
AdultWeekday         float64
AdultWeekend         float64
projectedDaysOpen      int64
NightSkiing_ac         int64
clusters               int64
dtype: object

In [8]:
dfo=df[['state']] # select object type columns

In [9]:
dfo

,state
0,Alaska
1,Alaska
2,Arizona
3,California
4,California
...,...
158,Wisconsin
159,Wyoming
160,Wyoming
161,Wyoming


In [11]:
dfx = pd.concat([df.drop(dfo, axis=1), pd.get_dummies(dfo)], axis=1)
dfx.head()

,Name,summit_elev,vertical_drop,trams,fastEight,fastSixes,fastQuads,quad,triple,double,...,state_Rhode Island,state_South Dakota,state_Tennessee,state_Utah,state_Vermont,state_Virginia,state_Washington,state_West Virginia,state_Wisconsin,state_Wyoming
0,Eaglecrest Ski Area,2600,1540,0,0,0,0,0,0,4,...,0,0,0,0,0,0,0,0,0,0
1,Hilltop Ski Area,2090,294,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,Sunrise Park Resort,11100,1800,0,0,0,1,2,3,1,...,0,0,0,0,0,0,0,0,0,0
3,Yosemite Ski & Snowboard Area,7800,600,0,0,0,0,0,1,3,...,0,0,0,0,0,0,0,0,0,0
4,Boreal Mountain Resort,7700,500,0,0,0,1,1,3,1,...,0,0,0,0,0,0,0,0,0,0


## Standardize the magnitude of numeric features

**<font color='teal'> Using sklearn preprocessing standardize the scale of the features of the dataframe except the name of the resort which we done't need in the dataframe for modeling, so it can be droppped here as well. Also, we want to hold out our response variable(s) so we can have their true values available for model performance review. Let's set `AdultWeekend` to the y variable as our response for scaling and modeling. Later we will go back and consider the `AdultWeekday`, `dayOpenLastYear`, and `projectedDaysOpen`. For now leave them in the development dataframe. </font>**

In [12]:
# first we import the preprocessing package from the sklearn library
from sklearn import preprocessing

# Declare an explanatory variable, called X,and assign it the result of dropping 'Name' and 'AdultWeekend' from the df
X = dfx.drop(['Name','AdultWeekend'], axis=1)

# Declare a response variable, called y, and assign it the AdultWeekend column of the df 
y = dfx.AdultWeekend 

# Here we use the StandardScaler() method of the preprocessing package, and then call the fit() method with parameter X 
scaler = preprocessing.StandardScaler().fit(X)

# Declare a variable called X_scaled, and assign it the result of calling the transform() method with parameter X 
X_scaled=scaler.transform(X) 

In [13]:
y

0      53.00000
1      34.00000
2      78.00000
3      47.00000
4      64.16681
         ...   
158    60.00000
159    48.00000
160    42.00000
161    59.00000
162    49.00000
Name: AdultWeekend, Length: 163, dtype: float64

## Split into training and testing datasets

**<font color='teal'> Using sklearn model selection import train_test_split, and create a 75/25 split with the y = `AdultWeekend`. We will start by using the adult weekend ticket price as our response variable for modeling.</font>**

In [14]:
# Import the train_test_split function from the sklearn.model_selection utility.  
from sklearn.model_selection import train_test_split

# Get the 1-dimensional flattened array of our response variable y by calling the ravel() function on y
y = y.ravel()

# Call the train_test_split() function with the first two parameters set to X_scaled and y 
# Declare four variables, X_train, X_test, y_train and y_test separated by commas 
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=1)

In [15]:
y_train

array([45.        , 47.        , 50.        , 40.        , 47.        ,
       73.        , 64.16681004, 45.        , 59.        , 64.16681004,
       45.        , 65.        , 69.        , 56.        , 52.        ,
       76.        , 79.        , 64.16681004, 84.        , 64.16681004,
       30.        , 64.16681004, 65.        , 70.        , 43.        ,
       44.        , 64.16681004, 64.16681004, 64.        , 64.16681004,
       78.        , 45.        , 80.        , 75.        , 53.        ,
       65.        , 72.        , 59.        , 50.        , 80.        ,
       63.2       , 49.        , 58.        , 92.        , 56.        ,
       55.        , 44.        , 43.        , 69.        , 52.        ,
       40.        , 50.        , 49.        , 20.        , 44.        ,
       49.        , 47.        , 62.        , 47.        , 84.        ,
       72.        , 65.        , 68.        , 59.        , 47.        ,
       48.        , 49.        , 50.        , 48.        , 69.  

Here we start the actual modeling work. First let's fit a multiple linear regression model to predict the `AdultWeekend` price.

# Guided Capstone Step 5. Modeling


This is the fifth step in the Data Science Method. In the previous steps you cleaned and prepared the datasets. Now it's time to get into the most exciting part: modeling! In this exercise, you'll build three different models and compare each model's performance. In the end, you'll choose the best model for demonstrating insights to Big Mountain management.



### **The Data Science Method**  


1.   Problem Identification 

2.   Data Wrangling 
  
3.   Exploratory Data Analysis 
 
4.   Pre-processing and Training Data Development

5.   **Modeling**
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

## Fit Models with a Training Dataset

**<font color='teal'> Using sklearn, fit the model on your training dataset.</font>**

#### Model 1

In [16]:
# all first model set
from sklearn import linear_model
from sklearn.metrics import explained_variance_score,mean_absolute_error
lm = linear_model.LinearRegression()
model = lm.fit(X_train,y_train)

**<font color='teal'> Predict on the testing dataset and score the model performance with the y_test set and the y-pred values. The explained variance is a measure of the variation explained by the model. This is also known as the R-squared value. </font>**

Hint: you will have to use the `predict()` method here as it's used in this [DSM article](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-modeling-56b4233cad1b) about modeling. 

In [17]:
# Make a variable called y_pred and assign it the result of calling predict() on our model variable with parameter X_test
y_pred = model.predict(X_test)

## Review Model Outcomes — Iterate over additional models as needed

In [18]:
# You might want to use the explained_variance_score() and mean_absolute_error() metrics.
# To do so, you will need to import them from sklearn.metrics. 
# You can plug y_test and y_pred into the functions to evaluate the model
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_error
evs=explained_variance_score(y_test, y_pred)

In [19]:
mae=mean_absolute_error(y_test, y_pred)

In [20]:
evs

-2.131705933616857e+22

In [21]:
mae

369411407487.19977

**<font color='teal'> Print the intercept value from the linear model. </font>**

Hint: our linear regression model `lm` has an attribute `intercept_` for the intercept

In [22]:
lm.intercept_

-92919433725.39832

**<font color='teal'> The intercept is the mean `AdultWeekend` price for all the resorts given the other characteristics. The addition or subtraction of each of the coefficient values in the regression are numeric adjustments applied to the intercept to provide a particular observation's value for the resulting `AdultWeekend` value. Also, because we took the time to scale our x values in the training data, we can compare each of the coeeficients for the features to determine the feature importances. Print the coefficient values from the linear model and sort in descending order to identify the top ten most important features.</font>** 


Hint: make sure to review the absolute value of the coefficients, because the adjustment may be positive or negative, but what we are looking for is the magnitude of impact on our response variable.

In [23]:
# You might want to make a pandas DataFrame displaying the coefficients for each state like so: 
pd.DataFrame(abs(lm.coef_), X.columns, columns=['Coefficient'])


,Coefficient
summit_elev,8.859873e-01
vertical_drop,3.583529e-01
trams,4.039781e+13
fastEight,8.877189e+13
fastSixes,9.570941e+12
fastQuads,6.324238e+10
quad,7.608066e+10
triple,1.347164e+11
double,1.604190e+11
surface,1.524507e+11


**<font color='teal'>You should see that the top ten important features are different states. However, the state is not something the managers at the Big Mountain Resort can do anything about. Given that we care more about actionable traits associated with ticket pricing, rebuild the model without the state features and compare the results. </font>**

Hint: Try to construct another model using exactly the steps we followed above. 

#### Model 2

In [24]:
# Declare an explanatory variable, called X,and assign it the result of dropping 'Name' and 'AdultWeekend' from the df
X = df.drop(['Name','state','AdultWeekend'], axis=1)

# Declare a response variable, called y, and assign it the AdultWeekend column of the df 
y = df.AdultWeekend 

# Here we use the StandardScaler() method of the preprocessing package, and then call the fit() method with parameter X 
scaler = preprocessing.StandardScaler().fit(X)

# Declare a variable called X_scaled, and assign it the result of calling the transform() method with parameter X 
X_scaled=scaler.transform(X) 

In [25]:
# Get the 1-dimensional flattened array of our response variable y by calling the ravel() function on y
y = y.ravel()

# Call the train_test_split() function with the first two parameters set to X_scaled and y 
# Declare four variables, X_train, X_test, y_train and y_test separated by commas 
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=1)

In [26]:
model = lm.fit(X_train,y_train)

In [27]:
y_pred = model.predict(X_test)

In [28]:
evs=explained_variance_score(y_test, y_pred)
mae=mean_absolute_error(y_test, y_pred)
print(evs)
print(mae)

0.7122381292424054
7.091554643560642


In [29]:
lm.intercept_

58.09111600617804

In [30]:
pd.DataFrame(abs(lm.coef_), X.columns, columns=['Coefficient'])

,Coefficient
summit_elev,2.744610e+00
vertical_drop,7.561088e-01
trams,2.664535e-15
fastEight,1.110223e-16
fastSixes,1.998401e-15
fastQuads,3.420982e-01
quad,1.295895e+00
triple,9.660889e-01
double,9.226994e-01
surface,1.476734e+00


**<font color='teal'> When reviewing our new model coefficients, we see `summit_elev` is now in the number two spot. This is also difficult to change from a management prespective and highly correlated with `base_elev` and `vertical_drop`.  This time, rebuild the model without the state features and without the `summit_elev` and without `base_elev`and compare the results. </font>**

#### Model 3

In [32]:
# Declare an explanatory variable, called X,and assign it the result of dropping 'Name' and 'AdultWeekend' from the df
X = df.drop(['Name','state','AdultWeekend','summit_elev'], axis=1)

# Declare a response variable, called y, and assign it the AdultWeekend column of the df 
y = df.AdultWeekend 

# Here we use the StandardScaler() method of the preprocessing package, and then call the fit() method with parameter X 
scaler = preprocessing.StandardScaler().fit(X)

# Declare a variable called X_scaled, and assign it the result of calling the transform() method with parameter X 
X_scaled=scaler.transform(X) 

In [33]:
y = y.ravel()

# Call the train_test_split() function with the first two parameters set to X_scaled and y 
# Declare four variables, X_train, X_test, y_train and y_test separated by commas 
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=1)
model = lm.fit(X_train,y_train)
y_pred = model.predict(X_test)
evs=explained_variance_score(y_test, y_pred)
mae=mean_absolute_error(y_test, y_pred)
print(evs)
print(mae)
print(lm.intercept_)

0.7115489103233086
7.020671189237886
58.007540444647645


In [34]:
pd.DataFrame(abs(lm.coef_), X.columns, columns=['Coefficient'])

,Coefficient
vertical_drop,7.237499e-01
trams,4.440892e-16
fastEight,4.440892e-16
fastSixes,1.332268e-15
fastQuads,3.988928e-01
quad,1.206044e+00
triple,8.043224e-01
double,9.313571e-01
surface,1.201908e+00
total_chairs,6.525553e-01


## Identify the Final Model

**<font color='teal'> Review the model performances in the table below and choose the best model for proving insights to Big Mountain management about what features are driving ski resort lift ticket prices. Type your choice in the final markdown cell — you will discuss this selection more in the next step of the guided casptone. </font>**

| Model | Explained Variance| Mean Absolute Error|Features Dropped|
| --- | --- | --- | --- |
| Model 1. | -2.131705933616857e+22 |369411407487.19977  |'base_elev'|
| Model 2. | 0.7122381292424054|7.091554643560642 |'state','base_elev'|
| Model 3. | 0.7115489103233086 |7.020671189237886 |'state','summit_elev','base_elev'|

Model Selection: Model 3